In [ ]:
!pip install geopandas

In [ ]:
!pip install folium

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd ## extension panda pour gérer des données géographiques
import folium ## permet de créer des cartes interactives
import requests ## pour faire des requêtes HTTP pour récupérer des données en ligne
from matplotlib import pyplot as plt ## partie de la bibliothèque Matplotlib utilisée pour faire des graphiques.
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler ## onctions de SciPy pour faire du clustering hiérarchique 

In [ ]:
from shapely.geometry import Point ## Shapely est une bibliothèque Python utilisée pour manipuler des objets géométriques (points, lignes, polygones) dans un contexte géospatial.

Ouverture du fichier sur les données sociodémographiques des IRIS

In [ ]:
revenus = pd.read_csv("revenus.csv", sep=";")
revenus.head()
## nous chargeons un fichier CSV nommé "revenus.csv" en utilisant la bibliothèque pandas et nous affichons les premières lignes du DataFrame résultant avec la méthode head().
## Ce tableau statistique de l'INSEE contient une colonne d'indentification géographique (IRIS) et de nombreuses colonnes de variables sur les revenus (médiane, déciles, quartiles, indices d’inégalité, parts de prestations, impôts, etc.) 

In [ ]:
revenus.columns = (
    revenus.columns
    .str.replace("^DISP_", "", regex=True)   # enlève le préfixe DISP_
    .str.replace("18$", "", regex=True)      # enlève le suffixe 18
    .str.lower()                             # met en minuscules
)
print(revenus.columns)
## Ici on nettoie les noms des colonnes du DataFrame en supprimant certains préfixes et suffixes spécifiques, et en convertissant tous les noms de colonnes en minuscules pour une meilleure lisibilité et cohérence. Ainsi on retir le préfixe "DISP_" et le suffixe "18" des noms de colonnes, puis on convertit tous les noms en minuscules. Enfin, on affiche les nouveaux noms de colonnes.

 Ici on nettoie les noms des colonnes du DataFrame en supprimant certains préfixes et suffixes spécifiques, et en convertissant tous les noms de colonnes en minuscules pour une meilleure lisibilité et cohérence. Ainsi on retir le préfixe "DISP_" et le suffixe "18" des noms de colonnes, puis on convertit tous les noms en minuscules. Enfin, on affiche les nouveaux noms de colonnes.

CAH
Faire un indice synthétique pour pouvoir représenter facilement les caractéristiques structurelles des quartiers sur une carte

In [ ]:
revenus.shape
## nombre de zones géographiques (lignes) et de variables (colonnes) dans le DataFrame revenus.

In [ ]:
revenus.isna().sum()

In [ ]:
# on retire la colonne IRIS qui correspond aux identifiants, 
# la colonne DISP_TP6018 (23% de NA) et la note de précaution
rev_cah = revenus.drop(columns=["iris", "tp60", "note", "d2", "d3", "d4", "d6", "d7", "d8"])
print(rev_cah.columns)

La commande revenus.isna().sum() sert à vérifier les valeurs manquantes dans ton DataFrame.revenus.isna() crée un tableau de la même taille que revenus où chaque cellule vaut :
- True si la valeur est manquante (NaN)
- False sinon
#.sum() fait la somme par colonne, donc on obtient le nombre de valeurs manquantes pour chaque colonne.

In [ ]:
# on vérifie qu'on n'a que des valeurs numériques
rev_cah.dtypes

In [ ]:
# imputation des valeurs manquantes
for col in rev_cah.columns:
    rev_cah[col] = rev_cah[col].fillna(rev_cah[col].median())

# vérification
rev_cah.isna().sum()

## Ce code remplace toutes les valeurs manquantes par la médiane de leur colonne et vérifie ensuite qu’il n’en reste plus. Cela prépare les données pour l’analyse ou le clustering.

In [ ]:
# normalisation
scaler = StandardScaler()
rev_scaled = scaler.fit_transform(rev_cah)

## La normalisation transforme les données pour que chaque colonne ait moyenne 0 et écart type 1, ce qui évite qu’une variable domine les autres et permet des analyses plus fiables.

In [ ]:
# CAH
Z = linkage(rev_scaled, method='ward')

plt.figure(figsize=(12, 6))
dendrogram(Z, truncate_mode="level", p=5)
plt.title("Dendrogramme CAH")
plt.show()

## Ce code réalise une Clustering Ascendant Hiérarchique (CAH) sur les données de revenus normalisées pour regrouper les IRIS aux caractéristiques similaires. La méthode de Ward est utilisée pour minimiser la variance à l’intérieur des clusters. Le dendrogramme affiché montre visuellement comment les IRIS sont regroupés et permet d’identifier le nombre de clusters pertinent pour analyser les profils socio-économiques des zones.*
## Expliquer comment on lit le dendrogramme

In [ ]:
last = Z[:, 2]  # distances des fusions
last_rev = last[::-1]  # inversé pour l’ordre croissant

plt.figure(figsize=(10, 5))
plt.plot(range(1, 16), last_rev[:15], marker='o')
plt.xlabel("Nombre de clusters")
plt.ylabel("Distance de fusion")
plt.title("Méthode du coude (1 à 15 clusters)")
plt.grid(True)
plt.show()

## On utilise la distance des fusions pour tracer une courbe et identifier le “coude”, c’est-à-dire le nombre de clusters où fusionner davantage devient peu utile. C’est une méthode visuelle et pratique pour déterminer le nombre optimal de clusters.
## L'axe x correspond au nombre de clusters et l'axe y à la distance de fusion (indique à quel point les clusters groupés sont différents). 

In [ ]:
# 7. Découpage en clusters
clusters = fcluster(Z, 5, criterion='maxclust') ## on choisit 5 clusters
revenus["cluster"] = clusters

# Résumé
print(revenus["cluster"].value_counts()) ## affiche le nombre d'IRIS dans chaque cluster

vars_to_summarize = ["tp60", "med", "rd", "gi", "pact", "ppat", "ppsoc"] ## on utilise seulement certaines variables pour le résumé

summary = revenus.groupby("cluster")[vars_to_summarize].mean() ## pour chaque variable, on calcule la moyenne dans chaque cluster
total = revenus[vars_to_summarize].mean()
summary_with_total = pd.concat([summary, total.to_frame().T], axis=0)
summary_with_total.index = list(summary.index) + ["Total"]

print(summary_with_total)

Dans un premier temps, ce code attribue chaque IRIS à un cluster (le clustering se fait par rapport à toutes les variables).
Ensuite, ce code de résume les caractéristiques socio-économiques des clusters obtenus par la classification. Il compte d’abord combien d’IRIS appartiennent à chaque groupe, puis calcule la moyenne de plusieurs variables représentatives (revenu médian, inégalités, part d’actifs, etc.) pour chaque cluster. Il ajoute enfin la moyenne globale du dataset pour permettre une comparaison. Le tableau final permet donc de comprendre le profil typique de chaque cluster par rapport à l’ensemble du territoire.

In [ ]:
summary = revenus.groupby("cluster").mean(numeric_only=True) ## calculer la moyenne des colonnes numériques pour chaque cluster
total = revenus.mean(numeric_only=True) ## calculer la moyenne générale de toutes les colonnes numériques
summary_with_total = pd.concat([summary, total.to_frame().T], axis=0) ## crée un DataFrame avec les moyennes par cluster + une ligne summplémentaire avec la moyenne générale
summary_with_total.index = list(summary.index) + ["Total"]

print(summary_with_total)

Après le clustering, chaque IRIS appartient à un groupe (cluster) ayant des caractéristiques similaires.

Le tableau summary te montre les moyennes par cluster pour les variables importantes (revenu médian, déciles, parts de prestations, etc.).

Ça permet de comparer les clusters entre eux : par exemple, quel cluster a les revenus les plus élevés, ou les inégalités les plus fortes.

La ligne Total représente la moyenne globale pour toutes les données, sans distinction de cluster. Elle sert de référence pour savoir si un cluster est au-dessus ou en dessous de la moyenne générale.

In [ ]:
cluster_order = revenus.groupby("cluster")["med"].median().sort_values()
print(cluster_order)
labels = ["tres_pauvre", "pauvre", "moyen", "riche", "tres_riche"]
mapping = {cluster: labels[i] for i, cluster in enumerate(cluster_order.index)}
mapping

In [ ]:
revenus["cluster_label"] = revenus["cluster"].map(mapping)
print(revenus["cluster_label"].value_counts())

Fusionner avec les contours des IRIS

In [ ]:
gdf_iris = gpd.read_file("contours-iris-pe.gpkg")

In [ ]:
gdf_iris = gdf_iris.merge(
    revenus,
    left_on="code_iris",
    right_on="iris",
    how="left"
)
print(gdf_iris.columns)

In [ ]:
print(gdf_iris.isna().sum())
gdf_iris.shape

Données démographiques sur les IRIS

In [ ]:
population = pd.read_csv("population.csv", sep=";")
population.head()

In [ ]:
population.columns = (
    population.columns
    .str.replace("^P21_", "", regex=True)   # enlève le préfixe DISP_
    .str.replace("^C21_", "", regex=True)      # enlève le suffixe 18
    .str.lower()                             # met en minuscules
)
print(population.columns)

In [ ]:
meta = pd.read_csv("meta_population.csv", sep=";")

# garder seulement les lignes correspondant à la variable IRIS
meta_iris = meta[meta["COD_VAR"] == "IRIS"]

# ne garder que le code et le nom
meta_iris = meta_iris[["COD_MOD", "LIB_MOD"]]

In [ ]:
meta_iris["COD_MOD"] = (
    meta_iris["COD_MOD"].astype(str)
                        .apply(lambda x: x[1:] if x.startswith("0") else x)
)

In [ ]:
population["iris"] = population["iris"].astype(str)
meta_iris["COD_MOD"] = meta_iris["COD_MOD"].astype(str)

In [ ]:
population = population.merge(
    meta_iris,
    left_on="iris",
    right_on="COD_MOD",
    how="left"
)

In [ ]:
population[["iris", "LIB_MOD"]].head()

Fusion des bases de données

In [ ]:
gdf_iris = gdf_iris.merge(
    population,
    left_on="code_iris",
    right_on="COD_MOD",
    how="left"
)
print(gdf_iris.columns)

In [ ]:
print(gdf_iris[["pop", "pop_fr"]].isna().sum())
gdf_iris.shape

In [ ]:
mapping_typ_iris = {
    "H": "habitat",
    "A": "activité",
    "D": "divers",
    "Z": "autre"
}

gdf_iris["type_iris_label"] = gdf_iris["type_iris"].map(mapping_typ_iris)
gdf_iris["type_iris_label"].value_counts()

Ouverture du fichier parcoursup

In [ ]:
parcoursup = "parcoursup.csv"

df = pd.read_csv(parcoursup, sep=";")
df.head()

In [ ]:
# Normaliser toutes les colonnes
df.columns = [
    col.lower()  # tout en minuscules
       .replace(' ', '_')       # espaces → _
       .replace("'", "_")       # apostrophes → _
       .replace('(', '')        # supprimer (
       .replace(')', '')        # supprimer )
       .replace(',', '')        # supprimer ,
       .replace('.', '')        # supprimer .
       .replace('…', '')
       .replace('é','e')        # accents
       .replace('è','e')
       .replace('à','a')
       .replace('ê','e')
       .replace('ç','c')
       .replace('%','percent')
       for col in df.columns
]

# Vérifier le résultat
print(df.columns)

Relier les IRIS à parcoursup

In [ ]:
df = df.rename(columns={"coordonnees_gps_de_la_formation": "coord_gps"})

df[['latitude', 'longitude']] = df['coord_gps'].str.split(',', expand=True)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Vérifier
df[['latitude','longitude']].head()

In [ ]:
df_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"   # CRS WGS84 pour des coordonnées GPS
)

df_points[['geometry']].head()

In [ ]:
# Transformation des IRIS en EPSG:4326
gdf_iris = gdf_iris.to_crs(epsg=4326)

# Vérifier le CRS
print(gdf_iris.crs)

In [ ]:
df_points = df_points.set_crs(4326)
gdf_iris = gdf_iris.set_crs(4326)

df_total = gpd.sjoin(
    df_points, 
    gdf_iris[['code_iris', 'nom_iris', 'geometry', 'nom_commune', 'type_iris', "med", "rd", "ppsoc", "cluster_label", "pop", "pop1117", "pop1824", "pop15p_cs3", "pop15p_cs5", "pop15p_cs6", "pop_imm"]], 
    how="left",
    predicate="within"
)

df_total.head()

In [ ]:
print(df_total.isna()[["pop", "pop_imm", "code_iris"]].sum())
print(df_total.shape)

In [ ]:
df_total["code_iris"].value_counts()

In [ ]:
df_total.columns

In [ ]:
df_total["selectivite"].value_counts()

In [ ]:
# 1. Nombre total de formations par IRIS
total_form = (
    df_total.groupby("code_iris")
      .size()
      .reset_index(name="nb_formations")
)

# 2. Nombre de formations sélectives par IRIS
selectives = (
    df_total[df_total["selectivite"] == "formation sélective"]
    .groupby("code_iris")
    .size()
    .reset_index(name="nb_form_sel")
)

# 3. Fusion des deux résultats
result = total_form.merge(selectives, on="code_iris", how="left")

# Les IRIS sans formation sélective → 0
result["nb_form_sel"] = result["nb_form_sel"].fillna(0).astype(int)

result.head(15)

In [ ]:
for col in df_total.columns:
    if "taux" in col.lower():
        print(col)

In [ ]:
df_total["taux_d’acces"].quantile([0.25, 0.333, 0.5, 0.75])

In [ ]:
(df_total["taux_d’acces"] < 50).mean()

In [ ]:
df_total["percent_d’admis_neo_bacheliers_boursiers"].quantile([0.25, 0.333, 0.5, 0.667, 0.75])

In [ ]:
(df_total["percent_d’admis_neo_bacheliers_boursiers"] < 30).mean()

In [ ]:
# créer les colonnes pour les formations très sélectives et avec un haut taux de boursiers
df_total["tres_select"] = df_total["taux_d’acces"] < 50
df_total["bcp_boursiers"] = df_total["percent_d’admis_neo_bacheliers_boursiers"] > 30

# compter par IRIS
result2 = df_total.groupby("code_iris")[["tres_select", "bcp_boursiers"]].sum().reset_index()

# fusionner aux autres colonnes créées 
result3 = result.merge(result2, on="code_iris", how="left")

# afficher
result3

Ajouter ces colonnes à la base sur les iris

In [ ]:
gdf_iris = gdf_iris.merge(result3, on="code_iris", how="left")
gdf_iris[["nb_formations", "nb_form_sel", "tres_select", "bcp_boursiers"]].head(15)

In [ ]:
# Remplacer les NaN par 0 pour les colonnes issues des données parcoursup
gdf_iris["nb_formations"] = gdf_iris["nb_formations"].fillna(0)

# Vérification
gdf_iris["nb_formations"].head(30)

In [ ]:
gdf_iris["nb_formations_cat"] = "0"

# sélectionner les valeurs > 0
mask = gdf_iris["nb_formations"] > 0

# créer les quantiles sur le reste
gdf_iris.loc[mask, "nb_formations_cat"] = pd.qcut(gdf_iris.loc[mask, "nb_formations"], 
                                            q=2, 
                                            labels=["Q1","Q2"])

gdf_iris.groupby("nb_formations_cat")["pop"].mean()

In [ ]:
# Centrer la carte sur Besançon
besancon_lon, besancon_lat = 6.025, 47.237
m = folium.Map(location=[besancon_lat, besancon_lon], zoom_start=13)

# Ajouter les polygones IRIS
folium.GeoJson(
    gdf_iris,
    name="IRIS",
    style_function=lambda x: {"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.2}
).add_to(m)

# Filtrer les points valides
df_points_valid = df_points.dropna(subset=['latitude', 'longitude'])

#Boucle sur les points valides
for idx, row in df_points_valid.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color="red",
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# Ajouter contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
#m

In [ ]:
# 1) Ajouter un code département à partir du code IRIS
gdf_iris["code_iris"] = gdf_iris["code_iris"].astype(str)
gdf_iris["code_dept"] = gdf_iris["code_iris"].str[:2]

# 2) Garder uniquement l'Île-de-France
idf_deps = ["75", "77", "78", "91", "92", "93", "94", "95"]
gdf_idf = gdf_iris[gdf_iris["code_dept"].isin(idf_deps)].copy()

print("Nombre d'IRIS en IDF :", gdf_idf.shape[0])

# 3) Palette de couleurs pour les types de quartiers (clusters)
cluster_colors = {
    "tres_pauvre": "#b30000",  # rouge foncé
    "pauvre":      "#fc8d59",  # orange
    "moyen":       "#fee08b",  # jaune
    "riche":       "#91bfdb",  # bleu clair
    "tres_riche":  "#4575b4",  # bleu foncé
}

def style_cluster(feature):
    label = feature["properties"].get("cluster_label")
    color = cluster_colors.get(label, "#cccccc")  # gris si NaN
    return {
        "fillColor": color,
        "color": "black",
        "weight": 0.3,
        "fillOpacity": 0.6,
    }

# 4) Filtrer les formations qui sont dans un IRIS IDF
idf_iris_codes = set(gdf_idf["code_iris"].astype(str).unique())

df_points_idf = df_total[
    df_total["code_iris"].astype(str).isin(idf_iris_codes)
].dropna(subset=["latitude", "longitude"])

print("Nombre de formations en IDF :", df_points_idf.shape[0])

# 5) Créer une carte centrée sur Paris
m = folium.Map(
    location=[48.8566, 2.3522],
    zoom_start=10,
    max_zoom=10,
    min_zoom=10,
    dragging=False,
    scrollWheelZoom=False,
    doubleClickZoom=False,
    zoomControl=False
)


# 6) Ajouter les polygones IRIS colorés selon le type de quartier
folium.GeoJson(
    gdf_idf,
    name="Quartiers (IRIS)",
    style_function=style_cluster,
    tooltip=folium.GeoJsonTooltip(
        fields=["nom_iris", "nom_commune", "cluster_label"],
        aliases=["IRIS", "Commune", "Type de quartier"],
        localize=True
    ),
).add_to(m)

# 7) Ajouter les formations en points rouges
for _, row in df_points_idf.iterrows():
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=1,
        color="red",
        fill=True,
        fill_opacity=0.8,
    ).add_to(m)

folium.LayerControl().add_to(m)
#m
print("ok")

In [ ]:
## hello 

In [ ]:
import statsmodels.formula.api as smf
# Construire la variable dépendante binaire Y = has_selective
# nb_form_sel = nombre de formations "formation sélective" dans l'IRIS (calculé plus haut)
# Si nb_form_sel est manquant (IRIS sans formation), on met 0.
gdf_iris["nb_form_sel"] = gdf_iris["nb_form_sel"].fillna(0)

# Variable binaire : 1 si au moins une formation sélective, 0 sinon
gdf_iris["has_selective"] = (gdf_iris["nb_form_sel"] > 0).astype(int)

print("Répartition de has_selective (0/1) :")
print(gdf_iris["has_selective"].value_counts(dropna=False))


In [ ]:
#Construire un dataframe d'estimation propre (sans géométrie)
# Un logit ne doit pas recevoir la colonne geometry (qui n'est pas numérique)
# On garde uniquement les colonnes utiles au modèle.
# Ici, le coeur du modèle : C(cluster_label)
# Et (optionnel) quelques contrôles démographiques/sociaux.
vars_cont = ["ppsoc", "med", "pop1824","pop15p_cs5"] 
# Colonnes catégorielles : cluster_label (obligatoire), type_iris_label (optionnel si dispo)
cat_vars = ["cluster_label"]
if "type_iris_label" in gdf_iris.columns:
    cat_vars.append("type_iris_label")

# Colonnes finales utilisées
cols_needed = ["has_selective"] + cat_vars + [v for v in vars_cont if v in gdf_iris.columns]

df_model = gdf_iris[cols_needed].copy()

# Nettoyage : on enlève les lignes avec NA sur les variables du modèle
df_model = df_model.dropna()

print("\nColonnes utilisées dans df_model :")
print(df_model.columns.tolist())
print("Taille de l'échantillon :", df_model.shape)

In [ ]:
# Spécifier le modèle logit (formule)
# C(cluster_label) indique à statsmodels que cluster_label est catégorielle (dummies auto)
# La catégorie de référence est choisie automatiquement (souvent ordre alphabétique).
# Si tu veux imposer une référence (ex. "tres_pauvre"), on peut le faire ensuite.
terms = ["C(cluster_label, Treatment(reference='moyen'))"]

# Ajout de type_iris_label si présent
if "type_iris_label" in df_model.columns:
    terms.append("C(type_iris_label)")

# Ajout des contrôles continus disponibles
for v in vars_cont:
    if v in df_model.columns:
        terms.append(v)

formula = "has_selective ~ " + " + ".join(terms)
print("\nFormule estimée :")
print(formula)

In [ ]:
#Estimer le logit par maximum de vraisemblance

logit_model = smf.logit(formula=formula, data=df_model)
logit_results = logit_model.fit()

print("\nRésumé du logit :")
print(logit_results.summary())


Interprétation des résultats du modèle logit :

Le tableau ci-dessus présente les résultats d’un modèle logit estimant la probabilité pour un quartier (IRIS) d’accueillir au moins une formation sélective. La variable dépendante est binaire et vaut 1 si l’IRIS comporte au moins une formation sélective, 0 sinon. Le modèle est estimé par maximum de vraisemblance sur un échantillon de 11 424 IRIS et a convergé correctement.

Qualité globale du modèle:

Le test du rapport de vraisemblance (LLR p-value < 10⁻¹²⁰) permet de rejeter très nettement l’hypothèse nulle selon laquelle l’ensemble des coefficients seraient nuls. Le modèle explique donc significativement la présence de formations sélectives. Le pseudo-R² s’élève à environ 5,8 %, ce qui est un niveau courant pour un modèle logit appliqué à des données spatiales et suggère que, bien que le modèle capte une part non négligeable des déterminants, une fraction importante de la localisation des formations reste expliquée par des facteurs non observés.

Lecture des coefficients associés aux types de quartiers:

Les coefficients estimés pour les types de quartiers sont exprimés en *log-odds* et doivent être interprétés relativement à une catégorie de référence. Dans ce modèle, les quartiers de type *moyen* constituent la catégorie de référence. Ainsi, chaque coefficient mesure l’écart de probabilité d’accueillir une formation sélective entre un type de quartier donné et un quartier moyen, toutes choses égales par ailleurs.

Un coefficient négatif indique que le type de quartier considéré a une probabilité plus faible que les quartiers moyens d’accueillir une formation sélective, tandis qu’un coefficient positif indique une probabilité plus élevée. Pour faciliter l’interprétation, ces coefficients peuvent être exponentiés afin d’obtenir des *odds ratios*, qui indiquent le facteur multiplicatif des chances relatives par rapport à la catégorie de référence.

Effet du type socio-économique du quartier:

Les quartiers *pauvres* présentent une probabilité significativement plus faible que les quartiers moyens d’accueillir une formation sélective. Cet écart est encore plus prononcé pour les quartiers *très pauvres*, dont la probabilité est fortement réduite par rapport à celle des quartiers moyens.
À l’autre extrémité de la distribution, les quartiers *très riches* se distinguent nettement des quartiers moyens par une probabilité significativement plus élevée d’accueillir une formation sélective. L’exponentiation du coefficient associé montre que les chances relatives d’implantation y sont plus de deux fois supérieures à celles observées dans les quartiers moyens. En revanche, les quartiers *riches* ne diffèrent pas significativement des quartiers moyens au seuil de 5 %, suggérant que l’avantage spatial se concentre principalement dans les quartiers les plus favorisés, et non de manière monotone avec le niveau de richesse.

Ces résultats révèlent une forte non-linéarité des inégalités socio-spatiales : les quartiers très riches concentrent une part disproportionnée de l’offre sélective, tandis que les quartiers pauvres et très pauvres sont nettement désavantagés.

Effet du type fonctionnel de l’IRIS:

Le modèle contrôle également pour le type fonctionnel des IRIS. À caractéristiques socio-économiques comparables, les IRIS à dominante résidentielle (*habitat*) présentent une probabilité plus faible d’accueillir une formation sélective que les IRIS d’activité, ce qui reflète une logique d’implantation liée à la présence d’infrastructures universitaires et de pôles d’enseignement.

Variables de contrôle démographiques et sociales:

Parmi les variables continues, la population âgée de 18 à 24 ans a un effet positif et fortement significatif : les quartiers comptant davantage de jeunes adultes ont une probabilité plus élevée d’accueillir des formations sélectives, ce qui est cohérent avec une logique de proximité à la population étudiante.

Le revenu médian apparaît avec un coefficient négatif conditionnellement aux clusters de quartiers, ce qui suggère que l’effet du niveau de revenu est déjà largement capturé par la typologie socio-économique globale. De même, la part des cadres et la part des prestations sociales dans le revenu ne présentent pas d’effet significatif une fois ces typologies prises en compte, indiquant une redondance informationnelle avec les clusters.


In [ ]:
#Interpréter en odds ratios (plus lisible que les log-odds)
# Les coefficients du logit sont en log-odds.
# Exp(coef) donne un odds ratio : multiplicateur des odds quand la variable augmente.
odds_ratios = np.exp(logit_results.params).sort_values(ascending=False)

print("\nOdds ratios (exp(coefficients)) :")
print(odds_ratios)

In [ ]:
#ça on comprend pas comment ça marche
#Calculer les probabilités prédites (p_hat) pour chaque IRIS du df_model
df_model["p_hat"] = logit_results.predict(df_model)

print("\nRésumé des probabilités prédites :")
print(df_model["p_hat"].describe())


In [ ]:
#Réinjecter les probabilités prédites dans gdf_iris pour cartographie
# On réassocie les p_hat à gdf_iris via l'index (car df_model vient de gdf_iris)
gdf_iris.loc[df_model.index, "p_hat"] = df_model["p_hat"]

print("\nColonne 'p_hat' ajoutée dans gdf_iris (extrait) :")
print(gdf_iris[["has_selective", "nb_form_sel", "cluster_label", "p_hat"]].head(10))